In [55]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from io import BytesIO

# 날짜와 etf ticker정보에 따라 api가져오는 함수 만들기

In [56]:
def getcsv(date_possible,ticker):
    url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    payload = {
        "locale": "ko_KR",
        "isuCd": ticker,
        "isuCd2" : "",  
        "param1isuCd_finder_secuprodisu1_1": "", 
        "trdDd": date_possible,
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false",
        "name": "fileDown",
        "url": "dbms/MDC/STAT/standard/MDCSTAT05001"
    }
    headers = {
        'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'
    }
    code = rq.post(url,payload,headers = headers).text

    down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    down_payload = {
        "code" : code
    }
    headers = {
        'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'
    }
    etf_individual = rq.post(down_url , down_payload , headers = headers)

    return pd.read_csv(BytesIO(etf_individual.content) , encoding = 'EUC_KR')




# etf정보 가져오기

In [57]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

engine = create_engine('mysql+pymysql://user:password@ip:3306/db')

etf_ticker = pd.read_sql("""
select * from etf_info
""",con = engine)
# 표준코드와 상장일 사용
# 6개월 계산 값이 상장일 보다 작으면 나가기

# etf의 모든 정보를 담는 데이터 프레임 생성

In [58]:
columns = ['종목코드 + 날짜' , '구성비중' , 'etf코드']
etf = pd.DataFrame(columns = columns)
print(etf)

Empty DataFrame
Columns: [종목코드 + 날짜, 구성비중, etf코드]
Index: []


# 3월달을 기준으로 1년단위로 etf가져오기

In [54]:
from datetime import date
from datetime import datetime, timedelta
import holidays
from itertools import chain
from tqdm import tqdm

kr_holidays = holidays.KR()

date_str = "20240331"

date_obj = datetime.strptime(date_str,"%Y%m%d").date()

for year in tqdm(range(15)):

    while (date_obj in kr_holidays or date_obj.weekday() >= 5):
        date_obj -= timedelta(days=1)
    date_possible = date_obj.strftime("%Y%m%d")

    for index, row in etf_ticker.iterrows():
        if row['상장일'] > date_obj:
            # print(row['상장일'])
            continue

        df = getcsv(date_possible,row['표준코드'])
        # print(row['표준코드'])
        # print(row['상장일'],row['표준코드'],df.head(5))
        filter = df['시가총액 구성비중'] > 0
        df = df[filter]

        total = df['시가총액 구성비중'].sum()
        if 95 < total < 105:
            df['종목코드 + 날짜'] = df['종목코드']+'/'+date_possible
            df['etf코드'] = row['표준코드']
            df = df.rename(columns = {'시가총액 구성비중' : '구성비중'})
            append_df = df[['종목코드 + 날짜' , '구성비중' , 'etf코드']]
            etf = pd.concat([etf,append_df])
            
            # print(etf.head())
            # df = df[['종목코드' , '시가총액 구성비중']]
            # print(df.head(5))
            #날짜마다 데이터 가져오기
            #섹터도 날짜마
            #df마다 종목의 부가 데이터 넣기
            #노드로 구
        
    date_obj = date(2023-year,3,31)
    time.sleep(1)
etf = etf.reset_index(drop=True)

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_26420\1259737880.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  etf = pd.concat([etf,append_df])
 13%|██████████▌                                                                    | 2/15 [56:16<6:05:49, 1688.41s/it]


ConnectionError: HTTPConnectionPool(host='data.krx.co.kr', port=80): Max retries exceeded with url: /comm/fileDn/download_csv/download.cmd (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002000527DF50>: Failed to establish a new connection: [WinError 10014] 호출에 대한 포인터 인수를 사용하려는 동안 시스템에서 잘못된 포인터 주소를 감지했습니다'))

In [60]:
pd.read_csv('C:/Users/user/.spyder-py3/퀀트/etf정보.csv')

,종목코드_날짜,구성비중,etf코드
0,005930/20240329,31.12,KR7069500007
1,000660/20240329,8.22,KR7069500007
2,005380/20240329,2.61,KR7069500007
3,068270/20240329,2.57,KR7069500007
4,005490/20240329,2.29,KR7069500007
...,...,...,...
180467,042670/20100331,0.37,KR7122090004
180468,047050/20100331,0.33,KR7122090004
180469,036460/20100331,0.32,KR7122090004
180470,001740/20100331,0.23,KR7122090004


In [ ]:
import time
time.sleep(2)
print("hi")

In [74]:
date_str = "20240430"

# 주어진 문자열을 datetime 객체로 변환
date_obj = datetime.strptime(date_str, "%Y%m%d").date()


getcsv(date_obj,"KR7451060008")

,종목코드,구성종목명,주식수(계약수),평가금액,시가총액,시가총액 구성비중
0,282330,BGF리테일,11.0,1415700.0,0,NaN
1,138930,BNK금융지주,379.0,2683320.0,0,NaN
2,001040,CJ,20.0,1904000.0,0,NaN
3,000120,CJ대한통운,16.0,2064000.0,0,NaN
4,097950,CJ제일제당,11.0,3520000.0,0,NaN
...,...,...,...,...,...,...
196,298050,효성첨단소재,4.0,1590000.0,0,NaN
197,298020,효성티앤씨,4.0,1434000.0,0,NaN
198,093370,후성,69.0,810750.0,0,NaN
199,081660,휠라홀딩스,57.0,2200200.0,0,NaN


In [68]:
etf_ticker[etf_ticker['표준코드'] == "KR7069500007"]

,표준코드,단축코드,종목명,상장일
0,KR7069500007,69500,삼성 KODEX200 증권상장지수투자신탁[주식],2002-10-14


In [59]:
etf_ticker

,표준코드,단축코드,종목명,상장일
0,KR7069500007,69500,삼성 KODEX200 증권상장지수투자신탁[주식],2002-10-14
1,KR7069660009,69660,키움 KOSEF 200 상장지수증권투자신탁[주식],2002-10-14
2,KR7091160002,91160,삼성 KODEX 반도체 증권상장지수투자신탁[주식],2006-06-27
3,KR7091170001,91170,삼성 KODEX 은행 증권상장지수투자신탁[주식],2006-06-27
4,KR7091180000,91180,삼성 KODEX 자동차 증권상장지수투자신탁[주식],2006-06-27
...,...,...,...,...
851,KR7480260009,480260,미래에셋 TIGER 27-04회사채(A+이상)액티브증권상장지수투자신탁(채권),2024-04-23
852,KR7480310002,480310,미래에셋 TIGER 글로벌온디바이스AI증권상장지수투자신탁(주식),2024-04-16
853,KR7480460005,480460,우리 WOORI 한국부동산TOP3플러스부동산상장지수투자신탁[재간접형],2024-04-30
854,KR7481050003,481050,삼성 KODEX 1년은행CD플러스액티브증권상장지수투자신탁[채권혼합-파생형](합성),2024-04-23


In [77]:
date_str = "20240430"

# 주어진 문자열을 datetime 객체로 변환
date_obj = datetime.strptime(date_str, "%Y%m%d").date()
date_possible = date_obj.strftime("%Y%m%d")

getcsv(date_possible,"KR7451060008")

AttributeError: 'str' object has no attribute 'strftime'